Import modules

In [1]:
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import cv2

%matplotlib inline

Using Theano backend.


Load data.

In [3]:
%load_ext autoreload
%autoreload 2
import sys
paths = ['../models', '../dataset', '../util']
for path in paths:
    if path not in sys.path:
        sys.path.append(path)
 
from bounding_boxes import *
from data_generators import get_data_with_bbox_coords
import dataset
from callbacks import ShowSegmentation

bboxes = get_bounding_boxes('../data')
bboxes = largest_bbox_per_image(bboxes)
train_data = dataset.train
val_data = dataset.validation
 
train_gen = get_data_with_bbox_coords(train_data, '../data/train', bboxes)
val_gen = get_data_with_bbox_coords(val_data, '../data/train', bboxes)
n_labels = len(dataset.labels)

In [67]:
def binning_generator(data_gen, n_bins = 10):
    #
    images, labels = data_gen.next()
    batch_size = len(images)
    label_x1 = np.zeros((batch_size, n_bins))
    label_x2 = np.zeros((batch_size, n_bins))
    label_y1 = np.zeros((batch_size, n_bins))
    label_y2 = np.zeros((batch_size, n_bins))
    for i,l, x1, x2, y1, y2 in zip(images, labels, label_x1, label_x2, label_y1, label_y2):
        shape = np.shape(i)
        bin_size = shape[1]//n_bins
        l[0] = l[0] // bin_size
        l[1] = l[1] // bin_size
        l[2] = l[2] // bin_size
        l[3] = l[3] // bin_size
        x1[l[0]] = 1 
        x2[l[1]] = 1 
        y1[l[2]] = 1 
        y2[l[3]] = 1 
    new_labels = np.array([label_x1, label_x2, label_y1, label_y2])
    yield images, new_labels

<generator object binning_generator at 0x11ae9f8c0>

Construct model 
Todo: softmax for binning training

In [3]:
def keras_model_conv():
    model = Sequential()
    
    model.add(BatchNormalization(axis=1,  input_shape=(3, 256, 256)))
    model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu',))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.50))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.50))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.50))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', 'mse'])

    return model

In [5]:
model = keras_model_conv()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_8 (BatchNorma (None, 3, 256, 256)   12          batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 16, 256, 256)  448         batchnormalization_8[0][0]       
____________________________________________________________________________________________________
batchnormalization_9 (BatchNorma (None, 16, 256, 256)  64          convolution2d_7[0][0]            
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 16, 128, 128)  0           batchnormalization_9[0][0]       
___________________________________________________________________________________________

In [ ]:
model.fit_generator(binning_generator(train_gen), samples_per_epoch=len(train_data),
                    nb_epoch=15, verbose=1,
                    callbacks=None,
                    validation_data=binning_generator(val_gen), nb_val_samples=len(val_data))